# **OBTENCIÓN DE DATOS DE AVIFAUNA - eBird API**

Este notebook obtiene los datos históricos de avistamientos de aves en la Presa Abelardo L. Rodríguez usando la API de eBird.

**Requisitos previos:**
- Ejecutar notebooks 1.0, 2.0 y 3.0 para tener el archivo `datos_hidrologicos_completos.csv`
- API key de eBird válida

**Salida:** 
- `../data/raw/avistamientos_ebird_raw.csv` (datos crudos de la API)
- `../data/processed/avistamientos_aves_presa.csv` (datos limpios)

## 1. Importar librerías y configurar rutas

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import json
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Progress bar
from tqdm.notebook import tqdm

# Configurar rutas siguiendo estructura Cookiecutter
project_dir = Path.cwd().parent
data_raw = project_dir / 'data' / 'raw'
data_processed = project_dir / 'data' / 'processed'

# Crear directorios si no existen
data_raw.mkdir(parents=True, exist_ok=True)
data_processed.mkdir(parents=True, exist_ok=True)

print(f"Directorio de datos raw: {data_raw}")
print(f"Directorio de datos processed: {data_processed}")

Directorio de datos raw: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\raw
Directorio de datos processed: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\processed


## 2. Configuración de eBird API

In [2]:
# Configuración de la API
EBIRD_API_KEY = "se6b2m6ll8ca"
LOC_ID = "L506196"  # Presa Abelardo L. Rodríguez
API_BASE_URL = "https://api.ebird.org/v2/data/obs"

# Headers requeridos por la API
HEADERS = {
    "X-eBirdApiToken": EBIRD_API_KEY
}

# Configuración de throttling y reintentos
SLEEP_BETWEEN_REQUESTS = 0.5  # segundos
MAX_RETRIES = 3
RETRY_BACKOFF = 2  # exponencial
RATE_LIMIT_WAIT = 60  # segundos para esperar si hay rate limit
CHECKPOINT_INTERVAL = 100  # Guardar progreso cada N requests

print("✓ Configuración de API completada")
print(f"  Location ID: {LOC_ID}")
print(f"  Sleep entre requests: {SLEEP_BETWEEN_REQUESTS}s")
print(f"  Max reintentos: {MAX_RETRIES}")

✓ Configuración de API completada
  Location ID: L506196
  Sleep entre requests: 0.5s
  Max reintentos: 3


## 3. Determinar rango de fechas a consultar

In [3]:
# Cargar datos hidrológicos procesados para obtener la fecha más antigua
file_hidro = data_processed / 'datos_hidrologicos_completos.csv'

if not file_hidro.exists():
    raise FileNotFoundError(
        f"❌ No se encontró {file_hidro}\n"
        "   Ejecuta primero el notebook 3.0-mcd-fusion-datos-hidrologicos.ipynb"
    )

df_hidro = pd.read_csv(file_hidro)
df_hidro['fecha'] = pd.to_datetime(df_hidro['fecha'])

# Filtrar fechas donde hay datos de presa (almacenamiento no es NaN)
df_con_presa = df_hidro[df_hidro['almacenamiento_hm3'].notna()]

# Obtener rango de fechas
fecha_minima = df_con_presa['fecha'].min().date()
fecha_maxima = datetime(2025, 10, 11).date()  # Fecha actual especificada

# Generar lista de fechas a consultar (desde la más reciente a la más antigua)
fechas_a_consultar = []
fecha_actual = fecha_maxima
while fecha_actual >= fecha_minima:
    fechas_a_consultar.append(fecha_actual)
    fecha_actual -= timedelta(days=1)

print("=" * 70)
print("RANGO DE FECHAS A CONSULTAR")
print("=" * 70)
print(f"\nFecha más antigua (datos de presa): {fecha_minima}")
print(f"Fecha más reciente: {fecha_maxima}")
print(f"Total de días a consultar: {len(fechas_a_consultar):,}")
print(f"\nTiempo estimado (con sleep de {SLEEP_BETWEEN_REQUESTS}s): "
      f"{len(fechas_a_consultar) * SLEEP_BETWEEN_REQUESTS / 3600:.2f} horas")
print("=" * 70)

RANGO DE FECHAS A CONSULTAR

Fecha más antigua (datos de presa): 1947-04-14
Fecha más reciente: 2025-10-11
Total de días a consultar: 28,671

Tiempo estimado (con sleep de 0.5s): 3.98 horas


## 4. Funciones auxiliares para requests a la API

In [4]:
def hacer_request_ebird(
    fecha: datetime.date,
    loc_id: str = LOC_ID,
    headers: dict = HEADERS,
    max_retries: int = MAX_RETRIES
) -> Optional[List[Dict]]:
    """
    Hace un request a la API de eBird para obtener avistamientos históricos.
    
    Args:
        fecha: Fecha a consultar
        loc_id: ID de la localización
        headers: Headers con API key
        max_retries: Número máximo de reintentos
    
    Returns:
        Lista de avistamientos o None si hay error
    """
    url = f"{API_BASE_URL}/{loc_id}/historic/{fecha.year}/{fecha.month}/{fecha.day}"
    
    for intento in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=30)
            
            # Request exitoso
            if response.status_code == 200:
                return response.json()
            
            # No hay datos para esta fecha
            elif response.status_code == 404:
                return []  # Lista vacía indica que no hay avistamientos
            
            # Rate limit excedido
            elif response.status_code == 429:
                print(f"\n⚠️  Rate limit alcanzado. Esperando {RATE_LIMIT_WAIT}s...")
                time.sleep(RATE_LIMIT_WAIT)
                continue
            
            # Otros errores HTTP
            else:
                print(f"\n⚠️  Error HTTP {response.status_code} para {fecha}: {response.text}")
                if intento < max_retries - 1:
                    wait_time = RETRY_BACKOFF ** intento
                    time.sleep(wait_time)
                    continue
                return None
        
        except requests.exceptions.Timeout:
            print(f"\n⚠️  Timeout para {fecha}. Reintento {intento + 1}/{max_retries}")
            if intento < max_retries - 1:
                time.sleep(RETRY_BACKOFF ** intento)
                continue
            return None
        
        except requests.exceptions.RequestException as e:
            print(f"\n❌ Error de red para {fecha}: {e}")
            if intento < max_retries - 1:
                time.sleep(RETRY_BACKOFF ** intento)
                continue
            return None
    
    return None


def extraer_campos_relevantes(avistamiento: Dict) -> Dict:
    """
    Extrae solo los campos relevantes de un avistamiento.
    
    Args:
        avistamiento: Diccionario con datos de avistamiento de la API
    
    Returns:
        Diccionario con campos relevantes
    """
    return {
        'fecha': avistamiento.get('obsDt'),
        'nombre_comun': avistamiento.get('comName'),
        'nombre_cientifico': avistamiento.get('sciName'),
        'cantidad': avistamiento.get('howMany'),
        'codigo_especie': avistamiento.get('speciesCode'),
        'es_exotica': avistamiento.get('exoticCategory', None)
    }


print("✓ Funciones auxiliares definidas")

✓ Funciones auxiliares definidas


## 5. Checkpoint: Verificar si hay progreso guardado

In [5]:
checkpoint_file = data_raw / 'checkpoint_avistamientos.json'
checkpoint_data_file = data_raw / 'avistamientos_checkpoint.csv'

# Verificar si hay checkpoint previo
fechas_ya_procesadas = set()
avistamientos_previos = []

if checkpoint_file.exists() and checkpoint_data_file.exists():
    print("📂 Checkpoint encontrado. Cargando progreso previo...")
    
    # Cargar checkpoint
    with open(checkpoint_file, 'r') as f:
        checkpoint = json.load(f)
    
    fechas_ya_procesadas = set(checkpoint.get('fechas_procesadas', []))
    
    # Cargar datos previos
    df_previo = pd.read_csv(checkpoint_data_file)
    avistamientos_previos = df_previo.to_dict('records')
    
    print(f"✓ Cargadas {len(fechas_ya_procesadas):,} fechas procesadas previamente")
    print(f"✓ Cargados {len(avistamientos_previos):,} avistamientos previos")
    
    # Filtrar fechas ya procesadas
    fechas_a_consultar = [
        f for f in fechas_a_consultar 
        if f.isoformat() not in fechas_ya_procesadas
    ]
    
    print(f"✓ Quedan {len(fechas_a_consultar):,} fechas por procesar")
else:
    print("📂 No se encontró checkpoint previo. Iniciando desde cero.")

print(f"\nTotal de requests a realizar: {len(fechas_a_consultar):,}")

📂 Checkpoint encontrado. Cargando progreso previo...
✓ Cargadas 3,100 fechas procesadas previamente
✓ Cargados 3,609 avistamientos previos
✓ Quedan 25,571 fechas por procesar

Total de requests a realizar: 25,571


## 6. Loop principal: Obtener datos de la API

In [6]:
# Listas para almacenar resultados
todos_avistamientos = avistamientos_previos.copy()
fechas_procesadas = fechas_ya_procesadas.copy()

# Contadores de estadísticas
stats = {
    'exitosos': 0,
    'sin_datos': 0,
    'errores': 0,
    'especies_unicas': set()
}

print("="*70)
print("INICIANDO OBTENCIÓN DE DATOS DE eBird API")
print("="*70)
print(f"\nFechas a procesar: {len(fechas_a_consultar):,}")
print(f"Checkpoint cada: {CHECKPOINT_INTERVAL} requests\n")

# Loop principal con barra de progreso
for idx, fecha in enumerate(tqdm(fechas_a_consultar, desc="Obteniendo avistamientos")):
    
    # Hacer request
    avistamientos = hacer_request_ebird(fecha)
    
    # Procesar resultados
    if avistamientos is None:
        stats['errores'] += 1
    elif len(avistamientos) == 0:
        stats['sin_datos'] += 1
    else:
        stats['exitosos'] += 1
        
        # Extraer campos relevantes
        for avistamiento in avistamientos:
            dato_limpio = extraer_campos_relevantes(avistamiento)
            todos_avistamientos.append(dato_limpio)
            
            # Actualizar especies únicas
            if dato_limpio['codigo_especie']:
                stats['especies_unicas'].add(dato_limpio['codigo_especie'])
    
    # Marcar fecha como procesada
    fechas_procesadas.add(fecha.isoformat())
    
    # Guardar checkpoint periódicamente
    if (idx + 1) % CHECKPOINT_INTERVAL == 0:
        # Guardar checkpoint
        checkpoint_data = {
            'fechas_procesadas': list(fechas_procesadas),
            'ultima_actualizacion': datetime.now().isoformat(),
            'stats': {
                'exitosos': stats['exitosos'],
                'sin_datos': stats['sin_datos'],
                'errores': stats['errores'],
                'total_avistamientos': len(todos_avistamientos),
                'especies_unicas': len(stats['especies_unicas'])
            }
        }
        
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint_data, f)
        
        # Guardar datos
        df_checkpoint = pd.DataFrame(todos_avistamientos)
        df_checkpoint.to_csv(checkpoint_data_file, index=False)
        
        print(f"\n💾 Checkpoint guardado en request {idx + 1}")
        print(f"   Avistamientos totales: {len(todos_avistamientos):,}")
        print(f"   Especies únicas: {len(stats['especies_unicas'])}")
    
    # Sleep para respetar rate limits
    time.sleep(SLEEP_BETWEEN_REQUESTS)

# Estadísticas finales
print("\n" + "="*70)
print("OBTENCIÓN COMPLETADA")
print("="*70)
print(f"\n✓ Requests exitosos: {stats['exitosos']:,}")
print(f"  Requests sin datos: {stats['sin_datos']:,}")
print(f"❌ Requests con error: {stats['errores']:,}")
print(f"\n🐦 Total de avistamientos: {len(todos_avistamientos):,}")
print(f"🦅 Especies únicas: {len(stats['especies_unicas'])}")
print("="*70)

INICIANDO OBTENCIÓN DE DATOS DE eBird API

Fechas a procesar: 25,571
Checkpoint cada: 100 requests



Obteniendo avistamientos:   0%|          | 0/25571 [00:00<?, ?it/s]


💾 Checkpoint guardado en request 100
   Avistamientos totales: 3,609
   Especies únicas: 0

💾 Checkpoint guardado en request 200
   Avistamientos totales: 3,740
   Especies únicas: 62

💾 Checkpoint guardado en request 300
   Avistamientos totales: 3,845
   Especies únicas: 84

💾 Checkpoint guardado en request 400
   Avistamientos totales: 3,922
   Especies únicas: 94

💾 Checkpoint guardado en request 500
   Avistamientos totales: 3,962
   Especies únicas: 95

💾 Checkpoint guardado en request 600
   Avistamientos totales: 4,054
   Especies únicas: 100

💾 Checkpoint guardado en request 700
   Avistamientos totales: 4,117
   Especies únicas: 108

💾 Checkpoint guardado en request 800
   Avistamientos totales: 4,144
   Especies únicas: 110

💾 Checkpoint guardado en request 900
   Avistamientos totales: 4,345
   Especies únicas: 117

💾 Checkpoint guardado en request 1000
   Avistamientos totales: 4,428
   Especies únicas: 120

💾 Checkpoint guardado en request 1100
   Avistamientos totales: 

## 7. Guardar datos crudos (raw)

In [7]:
# Convertir a DataFrame
df_raw = pd.DataFrame(todos_avistamientos)

# Guardar datos crudos
output_raw = data_raw / 'avistamientos_ebird_raw.csv'
df_raw.to_csv(output_raw, index=False)

print("✓ Datos crudos guardados")
print(f"  Archivo: {output_raw}")
print(f"  Registros: {len(df_raw):,}")
print(f"  Tamaño: {output_raw.stat().st_size / 1024 / 1024:.2f} MB")

# Limpiar archivos de checkpoint
if checkpoint_file.exists():
    checkpoint_file.unlink()
if checkpoint_data_file.exists():
    checkpoint_data_file.unlink()
print("\n🧹 Archivos de checkpoint eliminados")

print("\nPrimeros registros:")
display(df_raw.head(10))

✓ Datos crudos guardados
  Archivo: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\raw\avistamientos_ebird_raw.csv
  Registros: 4,854
  Tamaño: 0.30 MB

🧹 Archivos de checkpoint eliminados

Primeros registros:


,fecha,nombre_comun,nombre_cientifico,cantidad,codigo_especie,es_exotica
0,2025-10-11 11:05,Blue-winged Teal,Spatula discors,8.0,buwtea,NaN
1,2025-10-11 11:05,Mexican Duck,Anas diazi,18.0,mexduc,NaN
2,2025-10-11 11:05,Ruddy Duck,Oxyura jamaicensis,270.0,rudduc,NaN
3,2025-10-11 11:05,White-winged Dove,Zenaida asiatica,6.0,whwdov,NaN
4,2025-10-11 11:05,Mourning Dove,Zenaida macroura,70.0,moudov,NaN
5,2025-10-11 11:05,American Coot,Fulica americana,240.0,y00475,NaN
6,2025-10-11 11:05,Black-necked Stilt,Himantopus mexicanus,20.0,bknsti,NaN
7,2025-10-11 11:05,Killdeer,Charadrius vociferus,9.0,killde,NaN
8,2025-10-11 11:05,Solitary Sandpiper,Tringa solitaria,2.0,solsan,NaN
9,2025-10-11 11:05,Greater Yellowlegs,Tringa melanoleuca,5.0,greyel,NaN


## 8. Limpieza y procesamiento de datos

In [8]:
print("="*70)
print("LIMPIEZA Y PROCESAMIENTO DE DATOS")
print("="*70)

# Crear copia para procesar
df_processed = df_raw.copy()

# 1. Convertir fecha a datetime
df_processed['fecha'] = pd.to_datetime(df_processed['fecha'])
print(f"\n✓ Fechas convertidas a datetime")

# 2. Extraer solo la fecha (sin hora) para agregación
df_processed['fecha_solo'] = df_processed['fecha'].dt.date
print(f"✓ Columna fecha_solo creada")

# 3. Convertir cantidad a numérico (puede venir como string)
df_processed['cantidad'] = pd.to_numeric(df_processed['cantidad'], errors='coerce').fillna(1).astype(int)
print(f"✓ Cantidades convertidas a numérico")

# 4. Crear flag para especies exóticas
df_processed['es_exotica'] = df_processed['es_exotica'].notna()
print(f"✓ Flag de especies exóticas creado")

# 5. Verificar valores faltantes
print(f"\nValores faltantes por columna:")
missing = df_processed.isnull().sum()
for col, count in missing[missing > 0].items():
    print(f"  {col}: {count:,} ({count/len(df_processed)*100:.2f}%)")

# 6. Eliminar registros sin nombre científico (calidad de datos)
len_antes = len(df_processed)
df_processed = df_processed[df_processed['nombre_cientifico'].notna()]
len_despues = len(df_processed)
print(f"\n✓ Eliminados {len_antes - len_despues:,} registros sin nombre científico")

# 7. Ordenar por fecha
df_processed = df_processed.sort_values('fecha').reset_index(drop=True)
print(f"✓ Datos ordenados por fecha")

print("\n" + "="*70)
print(f"Registros después de limpieza: {len(df_processed):,}")
print("="*70)

LIMPIEZA Y PROCESAMIENTO DE DATOS


ValueError: time data "2003-12-17" doesn't match format "%Y-%m-%d %H:%M", at position 159. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## 9. Análisis exploratorio básico

In [ ]:
print("="*70)
print("ANÁLISIS EXPLORATORIO DE AVISTAMIENTOS")
print("="*70)

# Rango temporal
print(f"\n📅 Rango temporal:")
print(f"   Primera observación: {df_processed['fecha'].min().date()}")
print(f"   Última observación: {df_processed['fecha'].max().date()}")
print(f"   Años cubiertos: {df_processed['fecha'].dt.year.nunique()}")

# Estadísticas generales
print(f"\n🐦 Estadísticas generales:")
print(f"   Total de avistamientos: {len(df_processed):,}")
print(f"   Especies únicas: {df_processed['codigo_especie'].nunique()}")
print(f"   Total de individuos observados: {df_processed['cantidad'].sum():,}")
print(f"   Especies exóticas: {df_processed['es_exotica'].sum()}")

# Días con observaciones
dias_con_obs = df_processed['fecha_solo'].nunique()
dias_totales = (df_processed['fecha'].max().date() - df_processed['fecha'].min().date()).days + 1
cobertura = dias_con_obs / dias_totales * 100
print(f"\n📊 Cobertura temporal:")
print(f"   Días con observaciones: {dias_con_obs:,}")
print(f"   Días totales en rango: {dias_totales:,}")
print(f"   Cobertura: {cobertura:.2f}%")

print("\n" + "="*70)

### 9.1 Top 20 especies más observadas

In [ ]:
# Agrupar por especie y sumar avistamientos e individuos
especies_stats = df_processed.groupby(['nombre_comun', 'nombre_cientifico', 'codigo_especie']).agg({
    'cantidad': 'sum',
    'fecha': 'count'
}).reset_index()

especies_stats.columns = ['nombre_comun', 'nombre_cientifico', 'codigo_especie', 'total_individuos', 'num_avistamientos']
especies_stats = especies_stats.sort_values('total_individuos', ascending=False)

print("Top 20 especies por número total de individuos observados:\n")
display(especies_stats.head(20))

### 9.2 Especies exóticas identificadas

In [ ]:
especies_exoticas = df_processed[df_processed['es_exotica'] == True].groupby(
    ['nombre_comun', 'nombre_cientifico']
).agg({
    'cantidad': 'sum',
    'fecha': 'count'
}).reset_index()

especies_exoticas.columns = ['nombre_comun', 'nombre_cientifico', 'total_individuos', 'num_avistamientos']

if len(especies_exoticas) > 0:
    print(f"Se identificaron {len(especies_exoticas)} especies exóticas:\n")
    display(especies_exoticas)
else:
    print("No se identificaron especies exóticas en los datos.")

### 9.3 Estacionalidad: Avistamientos por mes

In [ ]:
# Agregar columnas de año y mes
df_processed['año'] = df_processed['fecha'].dt.year
df_processed['mes'] = df_processed['fecha'].dt.month
df_processed['mes_nombre'] = df_processed['fecha'].dt.month_name()

# Avistamientos por mes (promedio anual)
avistamientos_mes = df_processed.groupby('mes').size().reset_index(name='avistamientos')
avistamientos_mes['mes_nombre'] = pd.to_datetime(avistamientos_mes['mes'], format='%m').dt.month_name()

print("Avistamientos totales por mes (todos los años combinados):\n")
display(avistamientos_mes[['mes_nombre', 'avistamientos']])

### 9.4 Estadísticas descriptivas de cantidad de individuos

In [ ]:
print("Estadísticas de cantidad de individuos por avistamiento:\n")
display(df_processed['cantidad'].describe())

print(f"\nAvistamientos con mayor número de individuos:")
display(df_processed.nlargest(10, 'cantidad')[['fecha', 'nombre_comun', 'cantidad']])

## 10. Guardar dataset procesado

In [ ]:
# Seleccionar columnas finales para el dataset procesado
columnas_finales = [
    'fecha',
    'nombre_comun',
    'nombre_cientifico',
    'codigo_especie',
    'cantidad',
    'es_exotica'
]

df_final = df_processed[columnas_finales].copy()

# Guardar en data/processed
output_processed = data_processed / 'avistamientos_aves_presa.csv'
df_final.to_csv(output_processed, index=False)

print("="*70)
print("✅ DATASET PROCESADO GUARDADO EXITOSAMENTE")
print("="*70)
print(f"\nArchivo: {output_processed}")
print(f"Registros: {len(df_final):,}")
print(f"Columnas: {len(df_final.columns)}")
print(f"Tamaño: {output_processed.stat().st_size / 1024 / 1024:.2f} MB")
print(f"\nRango temporal: {df_final['fecha'].min().date()} a {df_final['fecha'].max().date()}")
print(f"\nColumnas incluidas:")
for i, col in enumerate(df_final.columns, 1):
    print(f"  {i}. {col}")

print("\n" + "="*70)
print("🎉 OBTENCIÓN Y PROCESAMIENTO DE DATOS DE AVIFAUNA COMPLETADO")
print("="*70)

## Resumen final

Este notebook ha completado exitosamente:

1. ✅ Configuración de la API de eBird con manejo robusto de errores
2. ✅ Determinación del rango de fechas basado en datos hidrológicos existentes
3. ✅ Obtención de datos históricos con:
   - Sistema de reintentos automáticos
   - Manejo de rate limits
   - Checkpoints para reanudar si se interrumpe
4. ✅ Limpieza y procesamiento de datos
5. ✅ Análisis exploratorio básico de biodiversidad
6. ✅ Guardado de datasets en formato Cookiecutter Data Science:
   - Raw: `data/raw/avistamientos_ebird_raw.csv`
   - Processed: `data/processed/avistamientos_aves_presa.csv`

Los datos están listos para análisis más profundos de:
- Diversidad de especies a lo largo del tiempo
- Correlación con niveles de agua de la presa
- Patrones estacionales de avifauna
- Impacto del ecosistema de la presa en la biodiversidad regional